<a href="https://colab.research.google.com/github/luisespriella9/AI-DeepLearning-Course/blob/develop/Churn_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.__version__

'2.3.0'

# Data Precprocessing

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.dropna()
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.drop(["RowNumber", "CustomerId", "Surname", "Exited"], axis=1)
y = dataset[["Exited"]]

## Encoding Categorical Data

Label Encoding Gender

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X["Gender"] = label_encoder.fit_transform(X["Gender"].values)

One hot encoding Geography

In [6]:
from sklearn.preprocessing import OneHotEncoder
hot_encoder = OneHotEncoder()
columns_transformed = hot_encoder.fit_transform(X[["Geography"]]).toarray()
ct_dataset = pd.DataFrame(columns_transformed, columns=hot_encoder.categories_)
X.drop(["Geography"], axis=1, inplace=True)
X = pd.concat([X, ct_dataset], axis=1)

In [7]:
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,"(France,)","(Germany,)","(Spain,)"
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


## Split dataset into Training set and Test set

In [8]:
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

## Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Build Artificial Neural Network

In [10]:
ann = tf.keras.models.Sequential()
# input layer
ann.add(tf.keras.layers.Dense(units=X_train.shape[1], activation='relu'))
# second layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Compile

In [11]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train ANN

In [12]:
ann.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
225/225 [==============================] - 0s 1ms/step - loss: 0.5603 - accuracy: 0.7307
Epoch 2/10
225/225 [==============================] - 0s 1ms/step - loss: 0.4412 - accuracy: 0.8012
Epoch 3/10
225/225 [==============================] - 0s 1ms/step - loss: 0.4190 - accuracy: 0.8136
Epoch 4/10
225/225 [==============================] - 0s 1ms/step - loss: 0.4028 - accuracy: 0.8264
Epoch 5/10
225/225 [==============================] - 0s 1ms/step - loss: 0.3875 - accuracy: 0.8360
Epoch 6/10
225/225 [==============================] - 0s 1ms/step - loss: 0.3769 - accuracy: 0.8407
Epoch 7/10
225/225 [==============================] - 0s 1ms/step - loss: 0.3687 - accuracy: 0.8485
Epoch 8/10
225/225 [==============================] - 0s 1ms/step - loss: 0.3626 - accuracy: 0.8492
Epoch 9/10
225/225 [==============================] - 0s 1ms/step - loss: 0.3582 - accuracy: 0.8533
Epoch 10/10
225/225 [==============================] - 0s 1ms/step - loss: 0.3543 - accuracy: 0.8564

# Evaluate Model

## Predict Test Results

In [13]:
y_pred = ann.predict(X_test)
y_pred_binary = (y_pred > 0.5)

## Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_binary)
cm

array([[1531,   54],
       [ 234,  181]])

In [15]:
accuracy_score(y_test, y_pred_binary)

0.856